In [7]:
# Steps to perform initially in Anaconda prompt
# conda create -n cvenv python=3.8 (create an environment "cvenv")
# conda activate cvenv (activate the environment)
# pip install opencv-python (install opencv)
# jupyter notebook (open jupyter notebook)


In [8]:
# Step - 1 - Import librarires
# cv2 library - image and video processing library (opencv - open source computer vision library)
# Numpy library - supports large multi-dimensional array for scientific computing (Numpy - Numerical Python)  

import cv2 
import time
import numpy as np 

In [9]:
#Step 2 - Capture the video through webcam

# FourCC is a 4-byte code used to specify the video file type (ex: XVID, MJPG, DIVX)
# fps - frame rate - how many frames do you want to display per second in your video
# iscolor - whether or not we are writing color frames to the file

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_vid = cv2.VideoWriter('output.avi',fourcc,20.0, (640,480),True)  # videowriter("filename",fourcc,fps,framesize,iscolor)

# Create a video capture object which is helpful to capture videos through webcam
# index passed is the id of the video capturing device to open. To open default camera using default backend just pass 0
capture = cv2.VideoCapture(0)

# system sleeps for 3 seconds before the webcam starts
time.sleep(3)
count = 0
frame = 0


In [10]:
# Capture the frame/background
for i in range(60):
     ret,frame = capture.read()  # ret = 1 if the video is captured; frame is the background image
frame = np.flip(frame,1) # flipcode > 0 - flips the image horizontally

In [11]:
# Object Tracking
# 1. Take each frame of the video
# 2. Convert from BGR to HSV color-space (we can use this to extract a colored object. 
#                                         In HSV, it is more easier to represent a color than RGB color-space.)
# 3. We threshold the HSV image for a range of blue color (or any chosen color)
# 4. Now extract the blue object alone, we can do whatever on that image we want.

In [ ]:
# Read each frame from webcam when the camera is open 
while(capture.isOpened()):
    ret,image = capture.read()  # ret = 1 if the video is captured; image is the frame captured by webcam
    if not ret:
        break
    count += 1
    image = np.flip(image, axis=1)
    # Color conversion from BGR to HSV 
    hsv_frame = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # H-Hue, S-Saturation, V-Value/brightness
    
     
    low_blue1 = np.array([80, 50, 20])
    upper_blue1 = np.array([100, 255, 255])
    mask1 = cv2.inRange(hsv_frame, low_blue1, upper_blue1) # Threshold the HSV image to get only blue colors
    

    lower_blue2 = np.array([100,50,20])
    upper_blue2 = np.array([135,255,255])
    mask2 = cv2.inRange(hsv_frame,lower_blue2,upper_blue2)  # Threshold the HSV image to get only blue colors
    mask1 = mask1 + mask2 # '+' means 'or' here , any blue shade as in mask 1 or mask 2 will be segmented and stored in mask 1
    
    # Removes noise in the image
    kernel = np.ones((3,3),np.uint8)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel) 
    
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, kernel)
    
    # Create an inverted mask to segment out the blue color from the frame
    mask2 = cv2.bitwise_not(mask1)
   
    # Segment the blue color part out of the frame using bitwise and with the inverted mask
    result1 = cv2.bitwise_and(image,image,mask=mask2)
   
    # Create image showing static background frame pixels only for the masked region
    result2 = cv2.bitwise_and(frame, frame, mask = mask1)
    
    # Using opencv, you can add or blend two images with the help of cv2.addWeighted() method.
    finalOutput = cv2.addWeighted(result1,1,result2,1,0)
    out_vid.write(finalOutput)
    cv2.imshow("Invisible_magic",finalOutput)
    cv2.waitKey(1)
cap.release()
out_vid.release()
cv2.destroyAllWindows()
